In [1]:
import os

import torch
import pytorch_lightning as pl

from electrolyte_fm.models import RoBERTa
from electrolyte_fm.data_modules import RobertaDataSet
from electrolyte_fm.utils.lr_schedule import RelativeCosineWarmup

/home/abhutani/electrolyte_fm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"

### Initialize Dataset

In [3]:
path = "/nfs/turbo/coe-venkvis/mist/realspace_v3_dev/"
tokenizer = "ibm/MoLFormer-XL-both-10pct"
mlm_probability = 0.15 
batch_size = 64
val_batch_size = 1


datamodule = RobertaDataSet(
    path=path,
    tokenizer=tokenizer,
    batch_size=batch_size,
    val_batch_size=val_batch_size
)

/home/abhutani/electrolyte_fm/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Initialize Model

In [5]:
vocab_size = datamodule.tokenizer.vocab_size
max_position_embeddings = 512
num_attention_heads = 12
num_attention_heads = 6
num_hidden_layers = 6
hidden_size = 768
intermediate_size = 768
relative_cosine_scheduler = lambda optimizer: RelativeCosineWarmup(optimizer, num_warmup_steps="beta2", num_training_steps=50_000)

model = RoBERTa(
    vocab_size=vocab_size,
    max_position_embeddings=max_position_embeddings,
    num_attention_heads=num_attention_heads,
    num_hidden_layers=num_hidden_layers,
    hidden_size=hidden_size,
    intermediate_size=intermediate_size,
    optimizer = torch.optim.AdamW,
    lr_schedule = relative_cosine_scheduler
)


### Initialize Trainer

In [6]:
# Some callbacks are defined for convinience
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

lr_monitor = LearningRateMonitor(logging_interval="step") # monitors and logs learning rate for schedulers during training

checkpoint_callback = ModelCheckpoint(save_last="link",
        filename="epoch={epoch}-step={step}-val_loss={val/loss_epoch:.2f}",
        monitor="val/loss_epoch",
        save_top_k=5,
        verbose=True,
        auto_insert_metric_name=False
    ) # saves the best model during training based on validation loss


In [11]:
trainer = pl.Trainer(
    precision = "16-mixed", # Combines FP32 and lower-bit floating points to reduce memory footprint and increase performance.
    strategy = "ddp_notebook", # Distributed Data Parallel training.
    use_distributed_sampler = False,  # Handled by DataModule (needed due to IterableDataset).
    limit_train_batches=500, 
    limit_val_batches=10, 
    max_epochs=1, 
    devices=torch.cuda.device_count(),
    callbacks=[lr_monitor, checkpoint_callback])

trainer.fit(model=model, datamodule=datamodule)


/home/abhutani/electrolyte_fm/.venv/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/abhutani/electrolyte_fm/.venv/lib/python3.11/s ...
Using 16bit Automatic Mixed Precision (AMP)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch

## Inference

In [16]:
for step, sample in enumerate(datamodule.val_dataloader()):
    print("Masked Molecule", datamodule.tokenizer.decode(sample['input_ids'].flatten()))
    mask = sample['labels'].flatten()!=-100
    labels = sample['labels'].flatten()[mask]
    print("Labels", datamodule.tokenizer.convert_ids_to_tokens(labels))
    pred = model(
        sample
    )
    pred = pred.logits[0].argmax(axis=1)[mask]
    pred = datamodule.tokenizer.convert_ids_to_tokens(pred)
    print("Labels", pred)
    print("_"*200)
    if step > 3:
        break

Masked Molecule <bos>CCS(=O<mask>(=[77Kr])NCC1=CN(CC<mask>CCOCCNC(=<mask>)CN2N=C<mask>C[AlH4-]C<mask>N3C(=O)C2=O)N=N1<eos>
Labels ['=', ')', 'O', 'O', 'O', '3', 'O', 'C']
Labels ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']
________________________________________________________________________________________________________________________________________________________________________________________________________
Masked Molecule <bos>COC(=O)C(<mask>C1=CN(CCN2CCC(N(C)C)C<mask><mask>N=N1)<mask>C(=<mask>)<mask>N1C=C(C<mask>N)=O)C[117Sn+4]<mask>O)<mask>C1=O<eos>
Labels ['C', 'N', '2', ')', 'N', 'O', 'C', '(', '(', '=', 'N']
Labels ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']
________________________________________________________________________________________________________________________________________________________________________________________________________
Masked Molecule <bos>NC<mask>=O)CCC(N<mask>(=O)(=O)C1=CC=C(Cl<mask>C<mask>C1<mask>C<mask><mask><mask>)N[C

## Attention Visualization

## Embedding Visualization